# Model Serving with Docker/Kubernetes and Tensorflow - MNIST Classification
---
*INPUT --> MODEL --> PREDICTION*

> **NOTE:** It is assumed that a model called *mnist* is already available in Hopsworks. An example of training a model for the *MNIST handwritten digit classification problem* is available in `Jupyter/end_to_end_pipelines/tensorflow/end_to_end_tensorflow.ipynb`

## Model Serving on [Hopsworks](https://github.com/logicalclocks/hopsworks)

![hops.png](../../../images/hops.png)

### Hopsworks Machine Learning (HSML) library

`hsml` is the library to interact with the Hopsworks Model Registry and Model Serving. The library makes it easy to export, manage and deploy models. To learn more about `hsml`, see the <a href="https://docs.hopsworks.ai/machine-learning-api/latest">Hopsworks Model Management</a> docs.

## Serve the MNIST classifier

#### Create a connection to Hopsworks

In [1]:
import hsml

# Connect with Hopsworks
conn = hsml.connection()

# Retrieve the model registry handle
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


### Query Model Registry for best Mnist Model

In [2]:
MODEL_NAME="mniste2e"

# Get the best version of the model
best_model = mr.get_best_model(MODEL_NAME, "accuracy", "max")

print('Model name: ' + best_model.name)
print('Model version: ' + str(best_model.version))
print(best_model.training_metrics)

Model name: mniste2e
Model version: 1
{'accuracy': '0.65625'}


### Create Deployment for the Trained Model

In [3]:
# Create serving instance
mnistclassifier = best_model.deploy()

After the serving have been created, you can find it in the Hopsworks UI by going to the "Deployments" tab. You can also use the class attributes or the `.describe()` method of a deployment object to describe access its metadata.

In [4]:
print("Deployment: " + mnistclassifier.name)
mnistclassifier.describe()

Deployment: mniste2e
{
    "artifact_version": 0,
    "batching_enabled": false,
    "created": "2022-05-18T14:21:38.217Z",
    "creator": "Admin Admin",
    "id": 5,
    "inference_logging": "ALL",
    "kafka_topic_dto": {
        "name": "CREATE",
        "num_of_partitions": 1,
        "num_of_replicas": 1
    },
    "model_name": "mniste2e",
    "model_path": "/Projects/demo_ml_meb10000/Models/mniste2e",
    "model_server": "TENSORFLOW_SERVING",
    "model_version": 1,
    "name": "mniste2e",
    "predictor": null,
    "predictor_resource_config": {
        "cores": 1,
        "gpus": 0,
        "memory": 1024
    },
    "requested_instances": 1,
    "serving_tool": "DEFAULT"
}


## Classify digits with the MNIST classifier

### Start Deployment

In [5]:
if mnistclassifier.get_state().status == "Stopped":
    mnistclassifier.start()

### Send Prediction Requests to the Deployed Model

For making inference requests you can use the `.predict()` method of the deployment metadata object.

In [6]:
for i in range(10):
    data = { "signature_name": "serving_default", "instances": [best_model.input_example] }
    predictions = mnistclassifier.predict(data)
    print(predictions)

{'predictions': [[0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]]}
{'predictions': [[0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]]}


## Monitor Prediction Requests and Responses using Kafka

In [7]:
from hops import kafka
from confluent_kafka import Producer, Consumer, KafkaError

Setup Kafka consumer and subscribe to the topic containing the prediction logs

In [8]:
TOPIC_NAME = mnistclassifier.inference_logger.kafka_topic.name

config = kafka.get_kafka_default_config()
config['default.topic.config'] = {'auto.offset.reset': 'earliest'}
consumer = Consumer(config)
topics = [TOPIC_NAME]
consumer.subscribe(topics)

Read the Kafka Avro schema from Hopsworks and setup an Avro reader

In [9]:
json_schema = kafka.get_schema(TOPIC_NAME)
avro_schema = kafka.convert_json_schema_to_avro(json_schema)

Read messages from the Kafka topic, parse them with the Avro schema and print the results

In [10]:
import json

PRINT_INSTANCES=False
PRINT_PREDICTIONS=True

for i in range(0, 5):
    msg = consumer.poll(timeout=5.0)
    if msg is not None:
        value = msg.value()
        try:
            event_dict = kafka.parse_avro_msg(value, avro_schema)
            
            print("serving: {}, version: {}, timestamp: {},\n"\
                  "        http_response_code: {}, model_server: {}, serving_tool: {}".format(
                       event_dict["modelName"],
                       event_dict["modelVersion"],
                       event_dict["requestTimestamp"],
                       event_dict["responseHttpCode"],
                       event_dict["modelServer"],
                       event_dict["servingTool"]))
            
            if PRINT_INSTANCES:
                print("instances: {}\n".format(event_dict["inferenceRequest"]))
            if PRINT_PREDICTIONS:
                print("predictions: {}\n".format(json.loads(event_dict["inferenceResponse"])["predictions"][0]))
                      
        except Exception as e:
            print("A message was read but there was an error parsing it")
            print(e)
    else:
        print("timeout.. no more messages to read from topic")

serving: mniste2e, version: 1, timestamp: 1652883746333,
        http_response_code: 200, model_server: TENSORFLOW_SERVING, serving_tool: DEFAULT
predictions: [0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]

serving: mniste2e, version: 1, timestamp: 1652883747147,
        http_response_code: 200, model_server: TENSORFLOW_SERVING, serving_tool: DEFAULT
predictions: [0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]

serving: mniste2e, version: 1, timestamp: 1652883747899,
        http_response_code: 200, model_server: TENSORFLOW_SERVING, serving_tool: DEFAULT
predictions: [0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]

serving: mniste2e, version: 1, timestamp: 1652883748556,
        http_response_code: 200, model_server: TENSORFLOW_SERVING, serving_tool: DEFAULT
predictions: [0.0, 0.0, 0.0, 0.0, 7.00596943e-14, 0.0, 0.0, 0.0, 0.0, 1.0]

serving: mniste2e, version: 1, timestamp: 1652883749242,
        http_response_code: 200, model_server: TENSORFL